In [44]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import statsmodels.api as sm
from math import sqrt

%matplotlib inline
mpl.style.use('classic')
'''
base_url = 'https://data.cityofboston.gov/resource/'
crime_bit = 'ufcx-3fdn.csv?$where DEPARTMENT NAME = '
employee_bit = '4swk-wcg8.csv?
#$limit=500&$offset=100' alter limit = 1000 to change number of records
crime_url = base_url + crime_bit
crime_df = pd.read_csv(crime_url)
employee_url = base_url + employee_bit
employee_df = pd.read_csv(employee_url)'''

crime_df = pd.read_csv('/Users/annaisaacson/Documents/workspace/Analytics_Local/Crime_Incident_Reports.csv')
empl_path = '/Users/annaisaacson/Documents/workspace/Analytics_Local/Employee_Earnings_Report_2014.csv'
employee_df = pd.read_csv(empl_path, dtype = {'NAME': str, 'TITLE': str} )
employee_df.rename(columns = {'DEPARTMENT NAME':'DeptName'}, inplace = True)
employee_df.rename(columns={'TOTAL EARNINGS': 'TotEarn'}, inplace = True)

def currency_to_float(val):
    val = float(val.replace('$', ''))
    return val

curr_cols = list(employee_df)[3:11]

for c in curr_cols:
    employee_df[c] = employee_df[c].map(currency_to_float)

,NAME,TITLE,DeptName,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN,TotEarn,ZIP
14067,"Pisani,Emiliabianca",Paraprofessional,Eliot K-8,728.62,0.00,0.00,0.00,0.00,0,0,728.62,02110
5111,"Carroll,William F",Teacher,BPS Boston Latin,70309.58,0.00,0.00,0.00,0.00,0,0,70309.58,01760
16441,"Araujo,Nancy B.",Teacher,Tobin K-8,85405.35,0.00,3750.00,0.00,0.00,0,0,89155.35,02124
4911,"Vina,Ela",Cluster Substitute,BPS Boston Arts Academy,18189.04,0.00,0.00,0.00,0.00,0,0,18189.04,37430
9386,"Thompson,Barbara A",Guidance Advisor(1),BPS McKinley Middle,97330.07,0.00,0.00,0.00,0.00,0,0,97330.07,01902
21444,"Lynn,Ryan Michael",Fire Fighter-Technician,Boston Fire Department,90631.67,13261.37,550.00,6890.81,2828.87,0,0,114162.72,02136
3026,"Pacitti,Robert",Clerk of Works(PMD Const &Rpr),Property Management,76393.50,0.00,3541.41,4320.12,0.00,0,0,84255.03,02127
8022,"Callahan,Jean M",Teacher,BPS Horace Mann,101115.51,0.00,0.00,0.00,0.00,0,0,101115.51,02021
5800,"Pai,Sung-Joon",Program Director (Basas 9A),BPS Charlestown High,111789.52,53.16,744.80,0.00,0.00,0,0,112587.48,02130
5459,"Cabral,Filomena",Registrar,BPS Burke High,101454.08,0.00,15154.70,0.00,0.00,0,0,116608.78,02368


In [47]:
# Which department gets the most overtime?
ot_pivot = pd.pivot_table(employee_df, values = 'OVERTIME', index = 'DeptName', aggfunc = 'sum')
ot_pivot.sort_values(ascending = False)

# Which person from each dept gets the most OT?
ot_who = pd.pivot_table(employee_df, index = ['DeptName', 'NAME'])
#ot_who = ot_who[ot_who.OVERTIME > 0]
#ot_who = ot_who.max(axis = 0, level = [1])
ot_who.sample(10)
#min = year_pivot.loc[year, 'actual_temp'].min()

,,DETAIL,INJURED,OTHER,OVERTIME,QUINN,REGULAR,RETRO,TotEarn
DeptName,NAME,,,,,,,,
Inspectional Services Dept,"Campbell,Michael Anthony",0,0,950.40,0.00,0,14101.75,0.00,15052.15
Boston Police Department,"Teal,Barbara J",0,0,743.37,34.45,0,10910.96,0.00,11688.78
Food & Nutrition Svc,"Pinto,Maria B.",0,0,0.00,0.00,0,42899.87,2161.88,45061.75
Election Division,"Piemonte,Michele",0,0,1411.84,11474.76,0,64618.09,0.00,77504.69
BPS Sumner Elementary,"Shaheen,Adam Francis",0,0,0.00,0.00,0,4236.93,0.00,4236.93
Elderly Commission,"Burdette,Arnett",0,0,0.00,0.00,0,41331.21,0.00,41331.21
Traffic Division,"Chow,Julio",0,0,996.41,3891.60,0,65177.60,0.00,70065.61
BPS Office Of Budget Mgmt,"Donkoh,Akua A.",0,0,0.00,0.00,0,21065.21,0.00,21065.21
BPS Special Education,"Farricy,Anne-Marie",0,0,0.00,0.00,0,79241.16,0.00,79241.16
